In [67]:
import pandas as pd
import numpy as np
from math import sqrt
from funcoesTermosol import *
import os

In [68]:
excel = pd.ExcelFile('Truss Setup.xlsx')
df_nodes = pd.read_excel(excel, 'Nodes')
df_incidences = pd.read_excel(excel, 'Elements')
df_loads = pd.read_excel(excel, 'Loads')
df_restrains = pd.read_excel(excel, 'Restrains')

In [69]:
amount_nodes = int(df_nodes.iloc[0, 3])
amount_incidence = int(df_incidences.iloc[0, 5])
amount_loads = int(df_loads.iloc[0, 4])
amount_restrains = int(df_restrains.iloc[0, 3])

df_node = df_nodes.iloc[:, 0:2].transpose()
df_incidence = df_incidences.iloc[:, 0:4]
df_load = df_loads.iloc[: , 0:3]
df_restrain = df_restrains.iloc[: , 0:2]

In [70]:
def freedom_matrix(node_amount, dimension=2):
    freedom_index = []
    for degree in range(dimension*node_amount):
        freedom_index.append(degree + 1)
    return pd.DataFrame(index=freedom_index, columns=freedom_index).fillna(0)

df_freedom = freedom_matrix(amount_nodes)

In [71]:
def element_length(node1, node2):
    n1 = df_node.iloc[:2, node1 - 1]
    n2 = df_node.iloc[:2, node2 - 1]
    return sqrt( (n2[0] - n1[0])**2 + (n2[1] - n1[1])**2 )

In [72]:
def matrix_sin_cos(node1, node2):
    l = element_length(node1, node2)
    n1 = df_node.iloc[:2, node1 - 1]
    n2 = df_node.iloc[:2, node2 - 1]
    c = (n2[0] - n1[0])**2/l
    s = (n2[1] - n1[1])**2/l
    freedom_index = [2*node1-1, 2*node1, 2*node2-1, 2*node2]
    data = [[  c**2,   c*s, -c**2,  -c*s],
            [   c*s,  s**2,  -c*s, -s**2],
            [ -c**2,  -c*s,  c**2,   c*s],
            [  -c*s, -s**2,   c*s,  s**2]]
    return pd.DataFrame(data, index=freedom_index, columns=freedom_index)

In [73]:
def Ke (node1, node2):
    msc = matrix_sin_cos(node1, node2)
    try:
        E = df_incidence[(matrix_incidence['Node 1'] == node1) & (df_incidence['Node 2'] == node2)]['E [Pa]']
        A = df_incidence[(matrix_incidence['Node 1'] == node1) & (df_incidence['Node 2'] == node2)]['A [m²]']
    except:
        E = df_incidence[(matrix_incidence['Node 1'] == node2) & (df_incidence['Node 2'] == node1)]['E [Pa]']
        A = df_incidence[(matrix_incidence['Node 1'] == node2) & (df_incidence['Node 2'] == node1)]['A [m²]']
    l = element_length(node1, node2)
    return ( E.values[0]*A.values[0]/l ) * msc

In [74]:
Ke_list = []
for element in range(amount_incidence):
    node1 = int(df_incidence.iloc[element,:].loc['Node 1'])
    node2 = int(df_incidence.iloc[element,:].loc['Node 2'])
    Ke_list.append(Ke(node1, node2))

In [75]:
def overlapping(matrix1, matrix2):
    overlap = lambda v1, v2: v1 + v2
    return matrix1.combine(matrix2, overlap).fillna(0)

In [76]:
df_global_rigidity = df_freedom.copy()
for Ke in Ke_list:
    df_global_rigidity += overlapping(df_global_rigidity, Ke)
df_global_rigidity

,1,2,3,4,5,6
1,2721600.0,4838400.0,0.0,0.0,-2721600.0,-4838400.0
2,4838400.0,42201600.0,0.0,-16800000.0,-4838400.0,-8601600.0
3,0.0,0.0,12600000.0,0.0,-12600000.0,0.0
4,0.0,-16800000.0,0.0,33600000.0,0.0,0.0
5,-2721600.0,-4838400.0,-12600000.0,0.0,27921600.0,4838400.0
6,-4838400.0,-8601600.0,0.0,0.0,4838400.0,8601600.0


In [77]:
df_restrains

,Node,1 = x 2 = y,Unnamed: 2,Restrains Amount
0,1,1,NaN,3.0
1,2,1,NaN,NaN
2,2,2,NaN,NaN


In [78]:

dimensions = 2
freedom_restrains = []
for restrain in range(amount_restrains):
    if df_restrains['1 = x               2 = y'][restrain] == 1:
        freedom_restrains.append( dimensions*df_restrain['Node'][restrain] - 1 )
    else:
        freedom_restrains.append( dimensions*df_restrain['Node'][restrain] )

print(freedom_restrains)    

[1, 3, 4]


In [79]:
new_rows = df_global_rigidity.index.difference(freedom_restrains).to_list()
new_cols = df_global_rigidity.columns.difference(freedom_restrains).to_list()
df_contours = df_global_rigidity.loc[new_cols, new_rows]
df_contours

,2,5,6
2,42201600.0,-4838400.0,-8601600.0
5,-4838400.0,27921600.0,4838400.0
6,-8601600.0,4838400.0,8601600.0


In [82]:
df_load

,Node,1 = x 2 = y,Load [N]
0,3,1,150
1,3,2,-100


In [91]:
df_global_forces = pd.DataFrame(index=new_cols, columns=['F']).fillna(0)
print(df_global_forces)
dimensions = 2
for load in range(amount_loads):
    if df_load['1 = x               2 = y'][load] == 1:
        df_global_forces['F'][dimensions*df_load['Node'][load] - 1 ] = df_load['Load [N]'][load] 
    else:
        df_global_forces['F'][dimensions*df_load['Node'][load]] = df_load['Load [N]'][load]

df_global_forces

   F
2  0
5  0
6  0


,F
2,0
5,150
6,-100


In [102]:

df_nodal_displacement = pd.DataFrame(np.linalg.solve(df_contours, df_global_forces))
df_nodal_displacement

,0
0,-0.000003
1,0.000008
2,-0.000019


In [94]:
df_global_rigidity.to_numpy()

array([[  2721600.        ,   4838400.        ,         0.        ,
                0.        ,  -2721600.        ,  -4838400.        ],
       [  4838400.        ,  42201600.00000001,         0.        ,
        -16800000.00000001,  -4838400.        ,  -8601600.        ],
       [        0.        ,         0.        ,  12600000.        ,
                0.        , -12600000.        ,         0.        ],
       [        0.        , -16800000.00000001,         0.        ,
         33600000.00000001,         0.        ,         0.        ],
       [ -2721600.        ,  -4838400.        , -12600000.        ,
                0.        ,  27921600.        ,   4838400.        ],
       [ -4838400.        ,  -8601600.        ,         0.        ,
                0.        ,   4838400.        ,   8601600.        ]])

In [96]:
df_incidence

,Node 1,Node 2,E [Pa],A [m²]
0,1,2,210000000000,0.0002
1,2,3,210000000000,0.0002
2,3,1,210000000000,0.0002
